# Analysis of Interferometric Wavefront Data

In this example, we will see how to use prysm to almost entirely supplant the software that comes with a commerical interferometer to analyze the wavefront of an optic.  We begin by importing the relevant classes and setting some aesthetics for matplotlib.

In [ ]:
from pathlib import Path

from prysm import Interferogram, FringeZernike, sample_files
from prysm.fringezernike import fit

from matplotlib import pyplot as plt
plt.style.use('bmh')

We point prysm to the file, create a new interferogram, mask it to a circular region 100 mm across, subtract piston, tip/tilt and power, and evalute the PV and RMS wavefront error. We also plot the wavefront to make sure all has gone well

In [ ]:
p = sample_files('dat')  # sample Zygo .dat file, will be downloaded on demand and saved locally
i = Interferogram.from_zygo_dat(p)
i.crop().mask('circle', 100).crop()
i.remove_piston_tiptilt_power()
print(i.pv, i.rms)
i.plot2d(clim=100)  # +/- 100 nm

The interferogram is cropped twice – once to enclose the valid data, then again to apply a mask centered on that region. For relatively conventional interferometry, you may want to stop here. If you want to use a different unit, that is easy enough,

In [ ]:
i.change_phase_unit('waves')
print(i.pv, i.rms)

There is no need to crop again since the outer bound has not changed. Perhaps you wish to evaluated the RMS within the 1 - 10 mm spatial periods,

In [ ]:
i.change_phase_unit('nm')
i.fill()
print(i.bandlimited_rms(1,10))

This value is derived from the PSD, so you must call fill first. Do not worry about the corners of the array containing data - it will be windowed out. If you do this on a part which has a central obscuration, the result will be incorrect. Otherwise, the value tends to agree to within +/- 5% of Zygo’s Mx ® software, though the authors of prysm do not believe they are calculated at all the same way.

If you wish to decompose the wavefront into Zernike polynomials, that is easy enough.

In [ ]:
# do this on data which has not been filled to avoid errors introduced by the fill value.
coefficients = fit(i.phase, terms=36, norm=True)
fz = FringeZernike(coefficients, dia=i.diameter, opd_unit=i.phase_unit, norm=True)
print(fz)

This print might be a bit daunting, one may prefer to see the top few terms by magnitude,

In [ ]:
print(fz.top_n(5))

or a barplot of all terms,

In [ ]:
fz.barplot_magnitudes(orientation='v')

The sample data has a ciruclar clear aperture, but if it had a central obscuration (such as transmitted wavefront data for a telescope) that would be easy to mask too.  PSD, and by extension bandlimited RMS values for data with annular support will be nonsensical.

In [ ]:
i.mask('invertedcircle', 25)
i.plot2d(clim=100)  # +/- 100 nm